# Welcome to LUSID!

In this tutorial we'll see how to use LUSID to perform the following task:

**<div align="center">As a UK asset manager, I run a single portfolio containing UK equities and investment grade vanilla bonds, and additionally maintain a cash position. The portfolio contains around 20 positions and trades around 5 positions a week. I want to view the asset allocation (equities vs bonds vs cash) on a daily basis.</div>**

In [ ]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import logging
logging.basicConfig(level = logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
#display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [ ]:
# Specify a unique scope and code to segregate data in this tutorial from others
module_scope = "FBNUniversity"
module_code = "Module1"

## 1. Populating the LUSID instrument master

We assume we have an `assets.csv` file exported from a securities database. We'll use this information to create valid instrument definitions for entities and bonds. Note LUSID is prepopulated with a basket of standard currencies, so we don't have to explicitly create the `GBP` currency instrument required to maintain our cash position.

In [ ]:
# Obtain the Instruments API
instruments_api=api_factory.build(la.InstrumentsApi)

# Read assets into pandas dataframe from securities file
assets_df = pd.read_csv("assets.csv", keep_default_na = False)
assets_df

In [ ]:
# Create a dictionary of instrument definitions from asset data
definitions = {}

# Iterate over each row in the assets dataframe
for index, asset in assets_df.iterrows():
    
    # Map identifier columns to case-sensitive LUSID identifier names       
    identifiers = {
        # Unique identifiers
        "Figi": lm.InstrumentIdValue(value = asset["figi"]),
        "ClientInternal": lm.InstrumentIdValue(value = asset["internal_id"]),
        # Non-unique identifiers
        "Isin": lm.InstrumentIdValue(value = asset["isin"]),
        "Ticker": lm.InstrumentIdValue(value = asset["ticker"])  
    }
                                       
    # Model equities
    if asset["security_type"] == "equity":
        # Create definitions
        definitions[asset["instrument_name"]] = lm.InstrumentDefinition(
            name = asset["instrument_name"],
            identifiers = identifiers,
            # SimpleInstrument type for now, soon to switch over to Equity
            definition = lm.SimpleInstrument(
                instrument_type = "SimpleInstrument",
                dom_ccy = asset["currency"],
                asset_class = "Equities",
                simple_instrument_type = "Equity",
            )
#             definition = lm.Equity(
#                 instrument_type = "Equity",
#                 dom_ccy = asset["currency"],
#                 identifiers = {}
#             )
        )
    # Model bonds
    elif asset["security_type"] == "govt bond":
        definitions[asset["instrument_name"]] = lm.InstrumentDefinition(
            name = asset["instrument_name"],
            identifiers = identifiers,
            definition = lm.Bond(
                instrument_type = "Bond",
                start_date = "2021-01-01",
                maturity_date = asset['maturity_date'],
                dom_ccy = asset["currency"],
                flow_conventions = lm.FlowConventions(
                    currency = asset["currency"],
                    payment_frequency = "6M",
                    day_count_convention = "ActualActual",
                    roll_convention = "NoAdjustment",
                    payment_calendars = [],
                    reset_calendars = [],
                    settle_days = 0,
                    reset_days = 0
                ),
                # Note that a bond instrument is unitised and the quantity specified in bond transactions
                principal = 1,
                coupon_rate = asset["coupon"]
            )
        )
    
# Upsert instruments to LUSID
upsert_instruments_response = instruments_api.upsert_instruments(request_body=definitions)

# Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

## 2. Creating a portfolio

In [ ]:
# Obtain the Transaction Portfolios API
transaction_portfolios_api=api_factory.build(la.TransactionPortfoliosApi)

# Create portfolio definition
definition = lm.CreateTransactionPortfolioRequest(
    display_name = "Module 1",
    code = module_code,
    base_currency = "GBP",
    # Must be before first transaction recorded
    created = "2021-12-01"
)

# Upsert portfolio to LUSID, making sure it's not already there
try:
    create_portfolio_response = transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = definition
    )
    # Confirm success
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])
  

## 3. Setting initial positions

We *could* load a stream of transactions by calling the UpsertTransactions API, but here we're going to set positions explicitly using the SetHoldings API, using data sourced from a file. We'll date them as effective 1 January 2022.

SetHoldings creates an 'adjustment transaction' for each position in a portfolio that does not impact the portfolio's cash balance. Note that any transaction booked into LUSID must specify at least one instrument identifier that enables LUSID to resolve it to a mastered instrument. In this example, we'll use FIGIs.

In [ ]:
# Obtain the Transaction Portfolios API (SetHoldings is an endpoint in this collection)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Read initial equity and bond position data into dataframe from file
positions_df = pd.read_csv('positions.csv')
# Create a virtual Cost column to multiply Quantity * Price for equities, and Price / 100 * Quantity for bonds
positions_df["Cost"] = positions_df.apply(
    lambda row: (row["Quantity"] * row["Price"] if row["Class"] == "Equity" 
                 else row["Price"] / 100 * row["Quantity"]), axis = 1) 
positions_df

In [ ]:
# Iterate over equities and bonds in the positions file, creating one adjustment transaction per row
holdings = [
    lm.AdjustHoldingRequest(
        # Resolve to mastered instruments using FIGI identifiers
        instrument_identifiers = {"Instrument/default/Figi": position["Figi"]},
        tax_lots = [
            lm.TargetTaxLotRequest(
                units = position["Quantity"],
                cost = lm.CurrencyAndAmount(
                    amount = position["Cost"],
                    currency = position["Currency"]
                ),
                portfolio_cost = position["Cost"],
                price = position["Price"])
        ]
    )
    for index, position in positions_df.iterrows()
]

# Create an initial cash position too
holdings.append(
    lm.AdjustHoldingRequest(
        # Resolve to GBP currency instrument prepopulated in LUSID
        instrument_identifiers = {"Instrument/default/Currency": "GBP"},
        tax_lots = [
            lm.TargetTaxLotRequest(
                units = 50000,
                cost = lm.CurrencyAndAmount(
                    amount = 50000,
                    currency = "GBP"
                ),
                price = 100
            )
        ]
    )
)

# Set holdings in LUSID
set_holdings_response = transaction_portfolios_api.set_holdings(
    scope = module_scope,
    code = module_code,
    # Make transactions effective from start of year
    effective_at = "2022-01-01",
    adjust_holding_request = holdings
)
print(f'{positions_df.shape[0]} equity/bond positions, plus cash, loaded into {module_scope}/{module_code} portfolio')

## 4. Calculating holdings

Now we can ask LUSID to calculate our official positions. For a portfolio containing a stream of transactions, LUSID automatically calculates instrument and cash positions from the stream of buys and sells. It also takes into account other economic activity, for example stock splits or dividends resulting from corporate actions.

Since we set our initial positions explicitly and no activity has occurred since, our holdings will be the same.

In [ ]:
# Obtain the Transaction Portfolios API (GetHoldings is an endpoint in this collection)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Get holdings for portfolio
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Decorate on instrument name property for clarity
    property_keys = ["Instrument/default/Name"]
)
# Transform API response to a pandas dataframe and show it
get_holdings_response_df = lusid_response_to_data_frame(get_holdings_response, rename_properties = True)
# Drop some noisy columns
get_holdings_response_df.drop(columns = [
    "instrument_scope", "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)"], inplace = True)
display(get_holdings_response_df)

## 5. Setting up valuation

### 5.1 Creating a recipe

Before we can value the portfolio, we need to create a recipe that tells LUSID where to find market data and which pricing model and methodology to use.

We'll be storing market data (prices) in the LUSID Quotes store later.

In [ ]:
# Obtain the Configuration Recipe API (note this is currently only available in the LUSID Python Preview SDK)
recipe_api = api_factory.build(la.ConfigurationRecipeApi)

# Create recipe definition
recipe = lm.ConfigurationRecipe(
    # Put the recipe in the same scope as the portfolio
    scope = module_scope,
    # Give the recipe a unique code in the scope
    code = f"{module_code}Recipe",
    description = "A recipe to value a portfolio containing equities and bonds",
    market = lm.MarketContext(
        market_rules = [
            # Look up pricing data for equities in the Quotes store
            lm.MarketDataKeyRule(
                # Look up equity quotes by Figi
                key = "Equity.Figi.*",
                supplier = "Lusid",
                data_scope = module_scope,
                quote_type = "Price",
                field = "mid",
                # Allow quotes to be valid for 2 years
                quote_interval = "2Y"
            ),
            # Look up pricing data for bonds in the Quotes store
            lm.MarketDataKeyRule(
                # Look up bond quotes by Figi
                key = "Credit.Figi.*",
                supplier = "Lusid",
                data_scope = module_scope,
                quote_type = "Price",
                field = "mid",
                # Allow quotes to be valid for 2 years
                quote_interval = "2Y"
            )
        ],
        options = lm.MarketOptions(
            default_supplier = "Lusid",
            # Ensure quotes are looked up by Figi
            default_instrument_code_type = "Figi",
            default_scope = module_scope,
        ),
    ),
    pricing = lm.PricingContext(
        # Force the SimpleStatic pricing model on bonds (this is already the default for equities).
        # Otherwise, bonds will be discounted.
        model_rules = [
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "SimpleStatic",
                instrument_type = "Bond"
            )
        ],
    )
)

# Upsert recipe to LUSID
upsert_recipe_response = (recipe_api.upsert_configuration_recipe(
        upsert_recipe_request = lm.UpsertRecipeRequest(
            configuration_recipe = recipe
        )
    )
)
print(f"Recipe upserted at {upsert_recipe_response.value}")

### 5.2 Specifying aggregation rules

We'll create a convenience function to call each time we want to value our portfolio that enables us to group assets in different ways. The function identifies the recipe to use and aggregates results by a set of metrics that present useful information.

In [ ]:
# Obtain the Aggregation API (note this is currently only available in the LUSID Python Preview SDK)
aggregation_api = api_factory.build(la.AggregationApi)

# Create convenience valuation function
def perform_valuation(date, group_by):
    # Create valuation request
    valuation_request = lm.ValuationRequest(
        # Choose recipe to use
        recipe_id = lm.ResourceId(scope = module_scope, code = f"{module_code}Recipe"),
        # Specify metrics (also known as queryable keys) to report useful information
        metrics = [
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/AssetClass", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Holding/default/Cost", "Sum"),
            lm.AggregateSpec("Valuation/PV/Amount", "Sum"),
            lm.AggregateSpec("Valuation/PV/Amount", "Proportion"),
            lm.AggregateSpec("Holding/default/Accrual", "Sum"),
        ],
        # Make grouping of results conditional on queryable key passed into function
        group_by = [group_by],
        # Identify portfolio to value
        portfolio_entity_ids = [lm.PortfolioEntityId(scope = module_scope, code = module_code)],
        # Make date of valuation conditional on effective at date passed into function
        valuation_schedule = lm.ValuationSchedule(effective_at = date),
    )
    
    # Get portfolio valuation
    val_data = aggregation_api.get_valuation(valuation_request = valuation_request).data
    
    # Turn valuation response into pandas dataframe
    vals_df = pd.DataFrame(val_data)
    # Turn valuation proportions into percentages
    vals_df["Proportion(Valuation/PV/Amount)"] = vals_df["Proportion(Valuation/PV/Amount)"] * 100
    # Give columns friendlier names
    vals_df.rename(
        columns = {
            "Instrument/default/Name": "Instrument",
            "Instrument/AssetClass": "Asset Class",
            "Sum(Valuation/PV/Amount)": "Present Value",
            "Proportion(Valuation/PV/Amount)": "% Portfolio",
            "Quotes/Price": "Price",
            "Holding/default/Units": "Units",
            "Sum(Holding/default/Cost)": "Cost",
            "Sum(Holding/default/Accrual)": "Interest Accrued"
        },
        inplace = True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    
    except:
        return vals_df

## 6. Valuing the portfolio on 1 January 2022

We created our portfolio and set initial positions effective from 1 January 2022. Let's value the portfolio at the end of that day.

### 6.1 Loading market data

The recipe requires market data (prices) and expects to find it in the LUSID Quotes store. The first task is therefore to load closing market prices for 1 January 2022.

In [ ]:
# Obtain the Quotes API
quotes_api = api_factory.build(la.QuotesApi)

# Load prices from market data vendor file
prices_df = pd.read_csv("prices.csv")
# Create a virtual Scale Factor column to enable valuation to scale properly: 1 for equities, 100 for bonds
prices_df["Scale Factor"] = prices_df["Class"].apply(lambda x: 1 if x == "Equity" else 100) 
display(prices_df)

In [ ]:
# Iterate over prices in the dataframe, creating one quote definition per row
quotes = {
    # Each quote must be upserted with an ephemeral key (uuid in this case), to track errors in the response
    str(uuid.uuid4()): lm.UpsertQuoteRequest(
        quote_id = lm.QuoteId(
            quote_series_id = lm.QuoteSeriesId(
                provider = "Lusid",
                instrument_id = price["Figi"],
                instrument_id_type = "Figi",
                quote_type = "Price",
                # Case sensitive: the field value must match that of the equivalent recipe field value exactly
                field = "mid",
            ),
            effective_at = to_date(price["Date"]),
        ),
        metric_value = lm.MetricValue(value = price["Price"], unit = price["Currency"]),
        # Bonds require a scale factor of 100
        scale_factor = price["Scale Factor"]
    )
    for row, price in prices_df.iterrows()
}

# Upsert quotes into LUSID
upsert_quotes_response = quotes_api.upsert_quotes(scope = module_scope, request_body = quotes)

if upsert_quotes_response.failed == {}:
    print(f"{len(upsert_quotes_response.values)} quotes successfully loaded into LUSID.")

else:
    print(f"Some failures occurred. {len(upsert_quotes_response.failed)} did not get loaded into LUSID.")

### 6.2 Valuing assets individually

To value each asset individually, we ask LUSID to group results by instrument name. Note that bonds are in the 'Credit' asset class.

In [ ]:
# Call our valuation function for close of day 1 Jan 2022, passing in the appropriate queryable key
valuation_df = perform_valuation("2022-01-01T17:00:00Z", "Instrument/default/Name")
valuation_df

### 6.3 Valuing by asset class

To aggregate up to equities vs bonds vs cash, we ask LUSID to group results by asset class.

In [ ]:
# Call our valuation function for close of day 1 Jan 2022, passing in the appropriate queryable key
valuation_df = perform_valuation("2022-01-01T17:00:00Z", "Instrument/AssetClass")
valuation_df

In [ ]:
# Create pie chart    
labels = [row["Asset Class"] for index, row in valuation_df.iterrows()]
values = [row["% Portfolio"] for index, row in valuation_df.iterrows()]
explode = [0] * len(labels)
index = labels.index("Equities")
explode[index] = 0.1
fig1, ax1 = plt.subplots(figsize = (10,10))
ax1.pie(values, explode = explode, labels = labels, autopct = '%1.1f%%', shadow = True, startangle = 90)
ax1.axis('equal')
plt.show()